# Dec 1, 2019 TF2 Keras Text binary cls 
* Name: Jikhan Jeong
* Ref: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [6]:
import tensorflow_datasets as tfds

In [4]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is available


* Split the training set into 60% and 40%, so we'll end up with 15,000 examples
* for training, 10,000 examples for validation and 25,000 examples for testing.

In [7]:
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

In [8]:
(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /home/jikhan.jeong/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [10]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=229, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [11]:
train_labels_batch

<tf.Tensor: id=230, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

*  Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples.
*  Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).

In [12]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [13]:
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [14]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=412, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

### Build Up Model 

In [15]:
model = tf.keras.Sequential()
model.add(hub_layer) ## Embedding layers
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Loss function and optimizer

In [16]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train the model
* Train the model for 20 epochs in mini-batches of 512 samples.   
* This is 20 iterations over all samples in the x_train and y_train tensors.   
* While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set  

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 9s 300ms/step - loss: 1.0809 - accuracy: 0.5170 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 7s 228ms/step - loss: 0.7083 - accuracy: 0.5657 - val_loss: 0.6663 - val_accuracy: 0.6046
Epoch 3/20
30/30 [==============================] - 7s 229ms/step - loss: 0.6465 - accuracy: 0.6357 - val_loss: 0.6244 - val_accuracy: 0.6594
Epoch 4/20
30/30 [==============================] - 7s 230ms/step - loss: 0.6064 - accuracy: 0.6766 - val_loss: 0.5930 - val_accuracy: 0.6946
Epoch 5/20
30/30 [==============================] - 7s 228ms/step - loss: 0.5769 - accuracy: 0.7096 - val_loss: 0.5647 - val_accuracy: 0.7224
Epoch 6/20
30/30 [==============================] - 7s 227ms/step - loss: 0.5448 - accuracy: 0.7398 - val_loss: 0.5359 - val_accuracy: 0.7461
Epoch 7/20
30/30 [==============================] - 7s 227ms/step - loss: 0.5147 - accuracy: 0.7655 - val_loss: 0.5075 - val_accuracy: 0.762

### Evaluate the model

In [18]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 6s - loss: 0.3212 - accuracy: 0.8639
loss: 0.321
accuracy: 0.864
